In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from scipy.io import loadmat
%matplotlib inline

In [3]:
path = '/gdrive/My Drive/ML:March2020/data/'

In [4]:
Movie_Ratings = pd.read_csv(path+'User_Movie_Ratings.csv')
Movie_Ratings.head()

,Movie Titles,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,Toy Story (1995),5,4,0,0,4,4,0,0,0,4,0,0,3,0,1,5,4,5,0,3,5,0,5,0,5,3,0,0,0,0,0,0,0,0,0,0,0,5,0,...,0,0,0,5,0,0,4,0,0,2,0,0,4,3,3,4,0,3,5,3,5,0,0,5,0,3,3,0,4,3,2,3,4,0,4,0,0,5,0,0
1,GoldenEye (1995),3,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,5
2,Four Rooms (1995),4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,3,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
3,Get Shorty (1995),3,0,0,0,0,0,5,0,0,4,0,5,5,0,0,5,0,3,4,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,0,0,0,0,2,0,0,0
4,Copycat (1995),3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0,0,0,3,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
Has_Rated = pd.read_csv(path+'Did_User_Rate_Movie.csv')
Has_Rated.head()

,Movie Title,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,Toy Story (1995),1,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,1,0,1,0,0,1,0,0
1,GoldenEye (1995),1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,Four Rooms (1995),1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Get Shorty (1995),1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
4,Copycat (1995),1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
num_movies = Movie_Ratings.shape[0]
num_users = Movie_Ratings.shape[1] - 1
print('Total Number of Movies = {}'.format(num_movies))
print('Total Number of Users = {}'.format(num_users))

Total Number of Movies = 1682
Total Number of Users = 943


In [7]:
Genres = 10

In [8]:
X =np.random.random(size=(num_movies,Genres))
X.shape

(1682, 10)

In [9]:
Xparam_num = X.size
Xparam_num

16820

In [10]:
Theta =np.random.random(size=(num_users,Genres))
Theta.shape

(943, 10)

In [11]:
params = np.concatenate((np.ravel(X), np.ravel(Theta)))

In [12]:
params.shape

(26250,)

In [13]:
Y=Movie_Ratings.drop(columns=['Movie Titles'])
Y.head()

,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,User 40,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,5,4,0,0,4,4,0,0,0,4,0,0,3,0,1,5,4,5,0,3,5,0,5,0,5,3,0,0,0,0,0,0,0,0,0,0,0,5,0,0,...,0,0,0,5,0,0,4,0,0,2,0,0,4,3,3,4,0,3,5,3,5,0,0,5,0,3,3,0,4,3,2,3,4,0,4,0,0,5,0,0
1,3,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,5
2,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,3,1,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0
3,3,0,0,0,0,0,5,0,0,4,0,5,5,0,0,5,0,3,4,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,0,0,0,0,0,2,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,0,0,0,3,0,0,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
Y.shape

(1682, 943)

In [15]:
R=Has_Rated.drop(columns=['Movie Title'])
R.head()

,User 1,User 2,User 3,User 4,User 5,User 6,User 7,User 8,User 9,User 10,User 11,User 12,User 13,User 14,User 15,User 16,User 17,User 18,User 19,User 20,User 21,User 22,User 23,User 24,User 25,User 26,User 27,User 28,User 29,User 30,User 31,User 32,User 33,User 34,User 35,User 36,User 37,User 38,User 39,User 40,...,User 904,User 905,User 906,User 907,User 908,User 909,User 910,User 911,User 912,User 913,User 914,User 915,User 916,User 917,User 918,User 919,User 920,User 921,User 922,User 923,User 924,User 925,User 926,User 927,User 928,User 929,User 930,User 931,User 932,User 933,User 934,User 935,User 936,User 937,User 938,User 939,User 940,User 941,User 942,User 943
0,1,1,0,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1,0,0,1,1,1,1,0,1,1,1,1,0,0,1,0,1,1,0,1,1,1,1,1,0,1,0,0,1,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
def cost(params, Y, R, Genres, reg_rate):
    Y = np.matrix(Y)  
    R = np.matrix(R)  
    num_movies = Y.shape[0]
    num_users = Y.shape[1]
    
    X_params = params[:Xparam_num]
    theta_params = params[Xparam_num:]

  
    X = np.matrix(np.reshape(X_params, (num_movies, Genres)))  
    Theta = np.matrix(np.reshape(theta_params, (num_users, Genres)))  
    
    
    error = np.multiply((X * Theta.T) - Y, R) 
    squared_error = np.power(error, 2)  
    J = (1. / 2) * np.sum(squared_error)
    
   
    J = J + ((reg_rate / 2) * np.sum(np.power(Theta, 2)))
    J = J + ((reg_rate / 2) * np.sum(np.power(X, 2)))

    
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape) 
    
    
    X_grad = (error * Theta) + (reg_rate * X)
    Theta_grad = (error.T * X) + (reg_rate * Theta)
    
    
    grad = np.concatenate((np.ravel(X_grad), np.ravel(Theta_grad)))
    
    return J, grad

In [17]:
J, grad = cost(params, Y, R, Genres, 3.5)
J

160970.8837539404

In [18]:
reg_rate = 3.5
from scipy.optimize import minimize

fmin = minimize(fun=cost, x0=params, args=(Y, R, Genres, reg_rate), method='CG', jac=True, options={'maxiter': 1000})
fmin

     fun: 43341.327536639306
     jac: array([0.10758261, 0.1114332 , 0.12778724, ..., 0.01137126, 0.00335202,
       0.01089956])
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 1496
     nit: 1000
    njev: 1496
  status: 1
 success: False
       x: array([ 1.16491284, -0.11658176,  0.35822714, ...,  0.41181031,
        1.14644153,  1.15522994])

In [19]:
X = np.matrix(np.reshape(fmin.x[:Xparam_num], (num_movies, Genres)))
Theta = np.matrix(np.reshape(fmin.x[Xparam_num:], (num_users, Genres)))

X.shape, Theta.shape

((1682, 10), (943, 10))

In [20]:
R = (R != 0)
Pred_Recommendations = round(np.multiply((X * Theta.T), R), 0)
Pred_Recommendations.shape

(1682, 943)

In [21]:
Pred_Recommendations= np.asarray(Pred_Recommendations)

In [22]:
Pred_Recommendations[:,0]

array([4., 3., 3., ..., 0., 0., 0.])

In [23]:
Movie_Ratings['User 1']

0       5
1       3
2       4
3       3
4       3
       ..
1677    0
1678    0
1679    0
1680    0
1681    0
Name: User 1, Length: 1682, dtype: int64

In [24]:
Y.shape[1]

943

In [25]:
from sklearn.metrics import accuracy_score

avg_acc=[]

for i in range(Y.shape[1]):
  acc = accuracy_score(Movie_Ratings['User '+str(i+1)], Pred_Recommendations[:,i])
  avg_acc.append(acc)
  print('Score of {}th User: {} %'.format(i+1, round(acc*100),2))

Score of 1th User: 92.0 %
Score of 2th User: 98.0 %
Score of 3th User: 98.0 %
Score of 4th User: 100.0 %
Score of 5th User: 94.0 %
Score of 6th User: 94.0 %
Score of 7th User: 87.0 %
Score of 8th User: 98.0 %
Score of 9th User: 99.0 %
Score of 10th User: 97.0 %
Score of 11th User: 94.0 %
Score of 12th User: 99.0 %
Score of 13th User: 83.0 %
Score of 14th User: 97.0 %
Score of 15th User: 97.0 %
Score of 16th User: 96.0 %
Score of 17th User: 99.0 %
Score of 18th User: 92.0 %
Score of 19th User: 99.0 %
Score of 20th User: 98.0 %
Score of 21th User: 95.0 %
Score of 22th User: 96.0 %
Score of 23th User: 96.0 %
Score of 24th User: 98.0 %
Score of 25th User: 99.0 %
Score of 26th User: 97.0 %
Score of 27th User: 99.0 %
Score of 28th User: 97.0 %
Score of 29th User: 99.0 %
Score of 30th User: 99.0 %
Score of 31th User: 99.0 %
Score of 32th User: 99.0 %
Score of 33th User: 100.0 %
Score of 34th User: 99.0 %
Score of 35th User: 99.0 %
Score of 36th User: 99.0 %
Score of 37th User: 98.0 %
Score of

In [26]:
Average_accuracy = sum(avg_acc)/Y.shape[1]
print('Average Accuracy in predictions for all users : {} %'.format(round(Average_accuracy*100),2))

Average Accuracy in predictions for all users : 97.0 %


In [27]:
Movie_Ratings['Movie Titles'][0]

'Toy Story (1995)'

In [28]:
new_ratings = np.zeros((1682, 1))

new_ratings[0] = 4
new_ratings[6] = 3
new_ratings[11] = 5
new_ratings[53] = 4
new_ratings[63] = 5
new_ratings[65] = 3
new_ratings[68] = 5
new_ratings[97] = 2
new_ratings[182] = 4
new_ratings[225] = 5
new_ratings[354] = 5

print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][0], str(int(new_ratings[0]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][6], str(int(new_ratings[6]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][11], str(int(new_ratings[11]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][53], str(int(new_ratings[53]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][63], str(int(new_ratings[63]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][65], str(int(new_ratings[65]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][68], str(int(new_ratings[68]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][97], str(int(new_ratings[97]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][182], str(int(new_ratings[182]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][225], str(int(new_ratings[225]))))
print('Rated {} with {} stars.'.format(Movie_Ratings['Movie Titles'][354], str(int(new_ratings[354]))))

Rated Toy Story (1995) with 4 stars.
Rated Twelve Monkeys (1995) with 3 stars.
Rated Usual Suspects, The (1995) with 5 stars.
Rated Outbreak (1995) with 4 stars.
Rated Shawshank Redemption, The (1994) with 5 stars.
Rated While You Were Sleeping (1995) with 3 stars.
Rated Forrest Gump (1994) with 5 stars.
Rated Silence of the Lambs, The (1991) with 2 stars.
Rated Alien (1979) with 4 stars.
Rated Die Hard 2 (1990) with 5 stars.
Rated Sphere (1998) with 5 stars.


In [29]:
Y = np.append(Y, new_ratings, axis=1)
R = np.append(R, new_ratings != 0,  axis=1)

In [30]:
Y.shape, R.shape

((1682, 944), (1682, 944))

In [31]:
num_movies = Y.shape[0]
num_users = Y.shape[1]

reg_rate = 0.5

X = np.random.random(size=(num_movies, Genres))
Theta = np.random.random(size=(num_users, Genres))

params = np.concatenate((np.ravel(X), np.ravel(Theta)))

X.shape, Theta.shape

((1682, 10), (944, 10))

In [32]:
reg_rate = 3.5
from scipy.optimize import minimize

fmin = minimize(fun=cost, x0=params, args=(Y, R, Genres, reg_rate), method='CG', jac=True, options={'maxiter': 1000})
fmin

     fun: 43377.06262370366
     jac: array([ 0.00768853,  0.0090794 , -0.00024194, ...,  0.00562944,
       -0.00338219,  0.00253724])
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 1494
     nit: 1000
    njev: 1494
  status: 1
 success: False
       x: array([0.77058132, 0.71247873, 0.84634046, ..., 0.59460792, 0.61230847,
       0.46975139])

In [33]:
fmin.x.shape

(26260,)

In [34]:
Xparam_num

16820

In [35]:
X = np.matrix(np.reshape(fmin.x[:Xparam_num], (num_movies, Genres)))
X.shape

(1682, 10)

In [36]:
Theta = np.matrix(np.reshape(fmin.x[Xparam_num:], (num_users, Genres)))
Theta.shape

(944, 10)

In [37]:
R = (R != 0)
Pred_Recommendations = np.multiply((X * Theta.T), R)
Pred_Recommendations.shape

(1682, 944)

In [39]:
R = (R != 1)
New_Recommendations = np.multiply((X * Theta.T), R)
my_recs = New_Recommendations[:, 943]

In [40]:
my_recs.shape

(1682, 1)

In [41]:
idx = np.argsort(my_recs, axis=0, )[::-1]
print(idx)

[[312]
 [180]
 [ 49]
 ...
 [ 97]
 [354]
 [  0]]


In [42]:
idx.shape

(1682, 1)

In [43]:
print("Top 10 movie predictions:")
for i in range(20):
    j = int(idx[i])
    print('Predicted rating of {0} for movie {1}.'.format(str((round(float(my_recs[j]),1))),Movie_Ratings['Movie Titles'][j]))

Top 10 movie predictions:
Predicted rating of 4.7 for movie Titanic (1997).
Predicted rating of 4.7 for movie Return of the Jedi (1983).
Predicted rating of 4.6 for movie Star Wars (1977).
Predicted rating of 4.6 for movie Raiders of the Lost Ark (1981).
Predicted rating of 4.6 for movie As Good As It Gets (1997).
Predicted rating of 4.5 for movie Empire Strikes Back, The (1980).
Predicted rating of 4.4 for movie Terminator 2: Judgment Day (1991).
Predicted rating of 4.4 for movie Braveheart (1995).
Predicted rating of 4.3 for movie Men in Black (1997).
Predicted rating of 4.3 for movie Terminator, The (1984).
Predicted rating of 4.2 for movie Apt Pupil (1998).
Predicted rating of 4.2 for movie Indiana Jones and the Last Crusade (1989).
Predicted rating of 4.2 for movie Back to the Future (1985).
Predicted rating of 4.2 for movie Conspiracy Theory (1997).
Predicted rating of 4.2 for movie Wrong Trousers, The (1993).
Predicted rating of 4.1 for movie Die Hard (1988).
Predicted rating of